In [3]:
import numpy as np
import math

def produce_pattern_database(goalPuzzle,N,numbers): # numbers is a list of elements we consider for producing pattern database
    for i in range(N):
        for j in range(N):
            b=False
            if goalPuzzle[i][j]!=0:
                for number in numbers:
                    if goalPuzzle[i][j]==number:
                        b=True
                if b == False:
                    goalPuzzle[i][j]=-1
    
    pattern_database=dict()
    produce_childs(goalPuzzle,pattern_database,0,N)
    return pattern_database


def produce_childs(goalPuzzle,pattern_database,distance,N):
    pattern_database[tuple(goalPuzzle.flatten())]=distance

    res=np.where(goalPuzzle == 0)
    coordinates_of_0=tuple(zip(res[0],res[1]))

    #left neighbor
    new=(coordinates_of_0[0][0]-1,coordinates_of_0[0][1])
    if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
        new_puzzle=np.copy(goalPuzzle)
        new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
        if tuple(new_puzzle.flatten()) not in pattern_database.keys():
            produce_childs(np.copy(new_puzzle),pattern_database,distance+1,N)
        

    #right neighbor
    new=(coordinates_of_0[0][0]+1,coordinates_of_0[0][1])
    if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
        new_puzzle=np.copy(goalPuzzle)
        new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
        if tuple(new_puzzle.flatten()) not in pattern_database.keys():
            produce_childs(np.copy(new_puzzle),pattern_database,distance+1,N)

    #upper neighbor
    new=(coordinates_of_0[0][0],coordinates_of_0[0][1]+1)
    if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
        new_puzzle=np.copy(goalPuzzle)
        new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
        if tuple(new_puzzle.flatten()) not in pattern_database.keys():
            produce_childs(np.copy(new_puzzle),pattern_database,distance+1,N)

    #lower neighbor
    new=(coordinates_of_0[0][0],coordinates_of_0[0][1]-1)
    if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
        new_puzzle=np.copy(goalPuzzle)
        new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
        if tuple(new_puzzle.flatten()) not in pattern_database.keys():
            produce_childs(np.copy(new_puzzle),pattern_database,distance+1,N)



In [ ]:

import numpy as np
import math
import heapq

#solving N-Puzzle using A* algorithm
# assumed that the input pazzle is valid
# (Enter 0 for empty cell)

# generating pattern database
pattern_database=0
numbers=0

def get_input():
    N=int(input('Enter the dimension of the puzzle: '))
    startPuzzle=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i == 0 and j == 0:
                temp=np.float64(input('Enter first entry of the starting puzzle matrix: '))
            else:
                temp=np.float64(input('Enter next entry: '))
            startPuzzle[i][j]=temp

    goalPuzzle=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i == 0 and j == 0:
                temp=np.float64(input('Enter first entry of the goal puzzle matrix: '))
            else:
                temp=np.float64(input('Enter next entry: '))
            goalPuzzle[i][j]=temp

    return N,startPuzzle,goalPuzzle


def Manhattan_distance_heuristic(x,goal):
    return abs(x[0]- goal[0][0])+ abs(x[1]- goal[0][1])

def Euclidean_distance_heuristic(x,goal):
    return math.sqrt((x[0]- goal[0][0])**2+(x[1]- goal[0][1])**2)

def f(puzzle,goalPuzzle,g,flag,N):
    if flag == 0: # manhattan distance 
        temp=0
        for i in range(N):
            for j in range(N):
                res=np.where(goalPuzzle == puzzle[i][j])
                temp+=Manhattan_distance_heuristic((i,j),tuple(zip(res[0],res[1])))
        
        return g[tuple(puzzle.flatten())]+temp
    
    elif flag == 1: # euclidean distance
        temp=0
        for i in range(N):
            for j in range(N):
                res=np.where(goalPuzzle == puzzle[i][j])
                temp+=Euclidean_distance_heuristic((i,j),tuple(zip(res[0],res[1])))
        
        return g[tuple(puzzle.flatten())]+temp
    
    else: # pattern database heuristic
        global numbers
        temp=np.copy(puzzle)
        for i in range(N):
            for j in range(N):
                b=False
                if temp[i][j]!=0:
                    for number in numbers:
                        if temp[i][j]==number:
                            b=True
                    if b == False:
                        temp[i][j]=-1

        global pattern_database
        return g[tuple(puzzle.flatten())]+pattern_database[tuple(temp.flatten())]

    
def A_star_for_N_Puzzle():

    flag=2
    n=int(input('Enter number of numbers you want to use in pattern database: ')) 
    
    global numbers
    numbers=list()
    for i in range(n):
        numbers.insert(0,int(input('Enter the number: ')))


    N,startPuzzle,goalPuzzle=get_input()

    global pattern_database
    pattern_database=produce_pattern_database(np.copy(goalPuzzle),N,numbers)

    g=dict() # distance from starting puzzle
    g[tuple(startPuzzle.flatten())]=0 

    closed=[]
    frontier=[(f(startPuzzle,goalPuzzle ,g,flag,N),tuple(startPuzzle.flatten()))] #priority queue - min heap

    path=dict()



    while True:

        if frontier == []:
            print('There is no path from starting puzzle to goal puzzle!')
            break

        f_score,node=heapq.heappop(frontier)
        closed.insert(0,node)
        node=np.array(node).reshape(N,N)

        if np.array_equal(node ,goalPuzzle):
            print('We use pattern database heuristic.')

            print('Number of steps required to get from staring puzzle to goal puzzle is: ',f_score)
            temp=[node]
            current=node
            while True:
                if np.array_equal(path[tuple(current.flatten())], startPuzzle):
                    break
                temp.insert(0,path[tuple(current.flatten())])
                current=path[tuple(current.flatten())] 

            print('Path from starting puzzle to goal puzzle:')
            for i in temp:
                print('change puzzle to ',i)
            
            break

        
        res=np.where(node == 0)
        coordinates_of_0=tuple(zip(res[0],res[1]))

        #left neighbor
        new=(coordinates_of_0[0][0]-1,coordinates_of_0[0][1])
        if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
            new_puzzle=np.copy(node)
            new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
            g[tuple(new_puzzle.flatten())]=g[tuple(node.flatten())]+1
            if tuple(new_puzzle.flatten()) not in closed and (f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())) not in frontier:
                heapq.heappush(frontier,(f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())))
                path[tuple(new_puzzle.flatten())]=node
            

        #right neighbor
        new=(coordinates_of_0[0][0]+1,coordinates_of_0[0][1])
        if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
            new_puzzle=np.copy(node)
            new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
            g[tuple(new_puzzle.flatten())]=g[tuple(node.flatten())]+1
            if tuple(new_puzzle.flatten()) not in closed and (f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())) not in frontier:
                heapq.heappush(frontier,(f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())))
                path[tuple(new_puzzle.flatten())]=node

        #upper neighbor
        new=(coordinates_of_0[0][0],coordinates_of_0[0][1]+1)
        if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
            new_puzzle=np.copy(node)
            new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
            g[tuple(new_puzzle.flatten())]=g[tuple(node.flatten())]+1
            if tuple(new_puzzle.flatten()) not in closed and (f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())) not in frontier:
                heapq.heappush(frontier,(f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())))
                path[tuple(new_puzzle.flatten())]=node

        #lower neighbor
        new=(coordinates_of_0[0][0],coordinates_of_0[0][1]-1)
        if new[0] >= 0 and new[1] >= 0 and new[0] < N and new[1] < N:
            new_puzzle=np.copy(node)
            new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]],new_puzzle[new[0]][new[1]]=new_puzzle[new[0]][new[1]],new_puzzle[coordinates_of_0[0][0]][coordinates_of_0[0][1]]
            g[tuple(new_puzzle.flatten())]=g[tuple(node.flatten())]+1
            if tuple(new_puzzle.flatten()) not in closed and (f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())) not in frontier:
                heapq.heappush(frontier,(f(new_puzzle,goalPuzzle,g,flag,N),tuple(new_puzzle.flatten())))
                path[tuple(new_puzzle.flatten())]=node




    

A_star_for_N_Puzzle()

